In [1]:
import tensorflow as tf
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

from argparse import ArgumentParser
from configparser import ConfigParser
from keras.models import Sequential
from keras.layers import Activation, Dense, AlphaDropout
from keras.callbacks import EarlyStopping
from keras.regularizers import l1, l2, l1_l2
from keras.constraints import max_norm
from keras.utils.generic_utils import get_custom_objects
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import compat
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # suppress Keras/TF warnings
compat.v1.logging.set_verbosity(compat.v1.logging.ERROR) # suppress Keras/TF warnings

In [2]:
tree = Signal('/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/NMSSM_cutflow_studies/presel/NMSSM_XYH_YToHH_6b_MX_700_MY_400_1M/ntuple.root')

/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/NMSSM_cutflow_studies/presel/NMSSM_XYH_YToHH_6b_MX_700_MY_400_1M/ntuple.root


In [3]:
id_sort = ak.argsort(tree.jet_signalId, axis=1, ascending=False)[:,:6]
tree.jet_signalId[id_sort]

<Array [[5, 4, 3, 2, 1, 0, ... 3, 2, 1, 0, -1]] type='569341 * var * int32'>

In [5]:
btag_sort = ak.argsort(tree.jet_btag, axis=1, ascending=False)[:,:6]

In [71]:
n_HX_bs = ak.sum(tree.jet_signalId == 0, axis=1).to_numpy() + ak.sum(tree.jet_signalId == 1, axis=1).to_numpy()

HX_b1_mask = tree.jet_signalId == 0
HX_b2_mask = tree.jet_signalId == 1

HX_found = (ak.sum(HX_b1_mask, axis=1) == 1) & (ak.sum(HX_b2_mask, axis=1) == 1)
print(ak.sum(HX_found))

HX_b1_mask = HX_b1_mask[HX_found]
HX_b2_mask = HX_b2_mask[HX_found]
HX_b1_mask, HX_b2_mask

409288


(<Array [[False, True, False, ... False, False]] type='409288 * var * bool'>,
 <Array [[False, False, ... False, False]] type='409288 * var * bool'>)

In [72]:
tree.jet_ptRegressed[HX_found][HX_b1_mask]

<Array [[135], [91.3], ... [217], [145]] type='409288 * var * float32'>

In [88]:
HX_b1_P4 = Particle(kin_dict={
    'pt' : ak.flatten(tree.jet_ptRegressed[HX_found][HX_b1_mask]),
    'eta' : ak.flatten(tree.jet_eta[HX_found][HX_b1_mask]),
    'phi' : ak.flatten(tree.jet_phi[HX_found][HX_b1_mask]),
    'm' : ak.flatten(tree.jet_mRegressed[HX_found][HX_b1_mask])
    })

HX_b2_P4 = Particle(kin_dict={
    'pt' : ak.flatten(tree.jet_ptRegressed[HX_found][HX_b2_mask]),
    'eta' : ak.flatten(tree.jet_eta[HX_found][HX_b2_mask]),
    'phi' : ak.flatten(tree.jet_phi[HX_found][HX_b2_mask]),
    'm' : ak.flatten(tree.jet_mRegressed[HX_found][HX_b2_mask])
    })

j1_P4 = Particle(kin_dict={
    'pt' : tree.jet_ptRegressed[~HX_found][:,0],
    'eta' : tree.jet_eta[~HX_found][:,0],
    'phi' : tree.jet_phi[~HX_found][:,0],
    'm' : tree.jet_mRegressed[~HX_found][:,0]
    })

j2_P4 = Particle(kin_dict={
    'pt' : tree.jet_ptRegressed[~HX_found][:,1],
    'eta' : tree.jet_eta[~HX_found][:,1],
    'phi' : tree.jet_phi[~HX_found][:,1],
    'm' : tree.jet_mRegressed[~HX_found][:,1]
    })

j3_P4 = Particle(kin_dict={
    'pt' : tree.jet_ptRegressed[~HX_found][:,2],
    'eta' : tree.jet_eta[~HX_found][:,2],
    'phi' : tree.jet_phi[~HX_found][:,2],
    'm' : tree.jet_mRegressed[~HX_found][:,2]
    })

j4_P4 = Particle(kin_dict={
    'pt' : tree.jet_ptRegressed[~HX_found][:,3],
    'eta' : tree.jet_eta[~HX_found][:,3],
    'phi' : tree.jet_phi[~HX_found][:,3],
    'm' : tree.jet_mRegressed[~HX_found][:,3]
    })

j5_P4 = Particle(kin_dict={
    'pt' : tree.jet_ptRegressed[~HX_found][:,4],
    'eta' : tree.jet_eta[~HX_found][:,4],
    'phi' : tree.jet_phi[~HX_found][:,4],
    'm' : tree.jet_mRegressed[~HX_found][:,4]
    })

j6_P4 = Particle(kin_dict={
    'pt' : tree.jet_ptRegressed[~HX_found][:,5],
    'eta' : tree.jet_eta[~HX_found][:,5],
    'phi' : tree.jet_phi[~HX_found][:,5],
    'm' : tree.jet_mRegressed[~HX_found][:,5]
    })

In [83]:
HX_b1_P4.pt, HX_b2_P4.pt

(<Array [135, 91.3, 73.3, ... 140, 217, 145] type='409288 * float32'>,
 <Array [40.4, 54.4, 70.2, ... 115, 104, 57.7] type='409288 * float32'>)

In [95]:
HX_features = np.column_stack((
    HX_b1_P4.pt.to_numpy(),
    HX_b1_P4.eta.to_numpy(),
    HX_b2_P4.pt.to_numpy(),
    HX_b2_P4.eta.to_numpy(),
    HX_b1_P4.deltaPhi(HX_b2_P4).to_numpy(),
    HX_b1_P4.deltaR(HX_b2_P4).to_numpy()
))

NO_features = np.row_stack((
    np.column_stack((
        j1_P4.pt.to_numpy(),
        j1_P4.eta.to_numpy(),
        j2_P4.pt.to_numpy(),
        j2_P4.eta.to_numpy(),
        j1_P4.deltaPhi(j2_P4).to_numpy(),
        j1_P4.deltaR(j2_P4).to_numpy()
        )),
    np.column_stack((
        j3_P4.pt.to_numpy(),
        j3_P4.eta.to_numpy(),
        j4_P4.pt.to_numpy(),
        j4_P4.eta.to_numpy(),
        j3_P4.deltaPhi(j4_P4).to_numpy(),
        j3_P4.deltaR(j4_P4).to_numpy()
        )),
    np.column_stack((
        j5_P4.pt.to_numpy(),
        j5_P4.eta.to_numpy(),
        j6_P4.pt.to_numpy(),
        j6_P4.eta.to_numpy(),
        j5_P4.deltaPhi(j6_P4).to_numpy(),
        j5_P4.deltaR(j6_P4).to_numpy()
        ))
    ))

n_events = HX_features.shape[0]
NO_features = NO_features[:n_events]
HX_features.shape, NO_features.shape

((409288, 6), (409288, 6))

In [101]:
features = np.row_stack((HX_features, NO_features))
n_inputs = features.shape[1]
features.shape

(818576, 6)

In [163]:
labels = np.row_stack((
    np.column_stack((
        np.ones(n_events),
        np.zeros(n_events)
    )),
    np.column_stack((
        np.zeros(n_events),
        np.ones(n_events)
    ))
))

labels.shape

(818576, 2)

In [164]:
labels.shape

(818576, 2)

In [102]:
# Hidden hyperparameters
hidden_activations = 'selu'
nodes              = [40,20]
nlayers            = len(nodes)

# Output hyperparameters
output_activation  = 'softmax'
output_nodes       = 2

optimizer_name     = 'nadam'
loss_function      = 'binary_crossentropy'
nepochs            = 50
batch_size         = 200

In [103]:
scaler = MinMaxScaler()
scaler.fit(features)
x = scaler.transform(features)

val_size = 0.10
x_train, x_val, y_train, y_val = train_test_split(x, labels, test_size=val_size)

param_dim = n_inputs

In [104]:
model = Sequential()

model.add(Dense(nodes[0], input_dim=param_dim, activation=hidden_activations, kernel_initializer='he_uniform'))
for i in range(1,nlayers):
    model.add(Dense(int(nodes[i]), activation=hidden_activations, kernel_constraint=max_norm(1.0), kernel_regularizer=l1_l2(), bias_constraint=max_norm(1.0)))
    # model.add(Dense(int(nodes[i]), activation=hidden_activations, kernel_initializer='he_uniform'))
    # model.add(Dense(int(nodes[i]), kernel_initializer='lecun_normal', activation=hidden_activations, kernel_constraint=max_norm(1.0), kernel_regularizer=l1_l2(), bias_constraint=max_norm(1.0)))
    # if bool(args.dropout): model.add(AlphaDropout(0.2)) 
model.add(Dense(output_nodes, activation=output_activation))

In [107]:
lr = 0.0001
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-7

es = EarlyStopping(monitor='loss', restore_best_weights=True, patience=5)
met = ['accuracy']

In [108]:
# optimizer = 'nadam'
optimizer = tf.keras.optimizers.Nadam(
    learning_rate=lr, beta_1=beta1, beta_2=beta2, epsilon=epsilon, name="nadam"
)

In [109]:
model.compile(loss=loss_function, 
              optimizer=optimizer, 
              metrics=met)

In [110]:
nn_info_list = [
    f"Input parameters:            {param_dim}\n",
    f"Optimizer:                   {optimizer}\n",
    f"Learning Rate:               {lr}\n",
    f"beta_1:                      {beta1}\n",
    f"beta_2:                      {beta2}\n",
    f"epsilon:                     {epsilon}\n",
    f"Loss:                        {loss_function}\n",
    f"Num epochs:                  {nepochs}\n",
    f"Batch size:                  {batch_size}\n",
    f"Num hidden layers:           {nlayers}\n",
    f"Input activation function:   {hidden_activations}\n",
    f"Hidden layer nodes:          {nodes}\n",
    f"Hidden activation functions: {hidden_activations}\n",
    f"Num output nodes:            {output_nodes}\n",
    f"Output activation function:  {output_activation.capitalize()}\n"]

for line in nn_info_list:
    print(line[:-1])

Input parameters:            6
Optimizer:                   <keras.optimizers.optimizer_v2.nadam.Nadam object at 0x7fa46e801a90>
Learning Rate:               0.0001
beta_1:                      0.9
beta_2:                      0.999
epsilon:                     1e-07
Loss:                        binary_crossentropy
Num epochs:                  50
Batch size:                  200
Num hidden layers:           2
Input activation function:   selu
Hidden layer nodes:          [40, 20]
Hidden activation functions: selu
Num output nodes:            2
Output activation function:  Softmax


In [111]:
# tf.config.run_functions_eagerly(True)

In [112]:
history = model.fit(x_train,
                    y_train, 
                    validation_data=(x_val, y_val), 
                    epochs=nepochs, 
                    batch_size=batch_size, 
                    callbacks=[es])

model.summary()

Epoch 1/50
3684/3684 [==============================] - 22s 5ms/step - loss: 1.0201 - accuracy: 0.7272 - val_loss: 0.6326 - val_accuracy: 0.7506
Epoch 2/50
3684/3684 [==============================] - 19s 5ms/step - loss: 0.6059 - accuracy: 0.7540 - val_loss: 0.5868 - val_accuracy: 0.7612
Epoch 3/50
3684/3684 [==============================] - 19s 5ms/step - loss: 0.5771 - accuracy: 0.7605 - val_loss: 0.5672 - val_accuracy: 0.7655
Epoch 4/50
3684/3684 [==============================] - 20s 5ms/step - loss: 0.5619 - accuracy: 0.7644 - val_loss: 0.5549 - val_accuracy: 0.7682
Epoch 5/50
3684/3684 [==============================] - 19s 5ms/step - loss: 0.5520 - accuracy: 0.7677 - val_loss: 0.5462 - val_accuracy: 0.7722
Epoch 6/50
3684/3684 [==============================] - 20s 5ms/step - loss: 0.5441 - accuracy: 0.7711 - val_loss: 0.5388 - val_accuracy: 0.7755
Epoch 7/50
3684/3684 [==============================] - 20s 5ms/step - loss: 0.5373 - accuracy: 0.7748 - val_loss: 0.5323 - val_ac

In [113]:
test_tree = Signal('/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/NMSSM_cutflow_studies/presel/NMSSM_XYH_YToHH_6b_MX_700_MY_400/ntuple.root')

/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/NMSSM_cutflow_studies/presel/NMSSM_XYH_YToHH_6b_MX_700_MY_400/ntuple.root


In [114]:
btag_sort = ak.argsort(test_tree.jet_btag, axis=1, ascending=False)
test_tree.jet_btag[btag_sort]

<Array [[0.997, 0.992, ... 0.0157, 0.0139]] type='55952 * var * float32'>

In [131]:
jet_index = ak.local_index(test_tree.jet_signalId[btag_sort])

# jet_label = (test_tree.jet_signalId[btag_sort] == -1) | (test_tree.jet_signalId[btag_sort] > 1)
jet_label = (test_tree.jet_signalId[btag_sort] == 0) | (test_tree.jet_signalId[btag_sort] == 1)

test_tree.jet_signalId[btag_sort][jet_index[jet_label]]
jet_label[:,0]*1
# jet_label

<Array [0, 1, 0, 0, 0, 0, ... 0, 0, 0, 0, 1, 1] type='55952 * int64'>

In [136]:
test_tree.jet_ptRegressed[btag_sort][:,i].to_numpy()

array([ 57.357674, 190.21924 , 292.987   , ..., 203.44594 , 132.53278 ,
       108.16328 ], dtype=float32)

In [148]:
test_labels = []
test_outputs = []

for i in range(5):
    for j in range(i+1,6):
        jet_1 = Particle(kin_dict={
            'pt' : test_tree.jet_ptRegressed[btag_sort][:,i].to_numpy(),
            'eta' : test_tree.jet_eta[btag_sort][:,i].to_numpy(),
            'phi' : test_tree.jet_phi[btag_sort][:,i].to_numpy(),
            'm' : test_tree.jet_mRegressed[btag_sort][:,i].to_numpy()
            })

        jet_2 = Particle(kin_dict={
            'pt' : test_tree.jet_ptRegressed[btag_sort][:,j].to_numpy(),
            'eta' : test_tree.jet_eta[btag_sort][:,j].to_numpy(),
            'phi' : test_tree.jet_phi[btag_sort][:,j].to_numpy(),
            'm' : test_tree.jet_mRegressed[btag_sort][:,j].to_numpy()
            })
        
        
        jet_features = np.column_stack((
            jet_1.pt,
            jet_1.eta,
            jet_2.pt,
            jet_2.eta,
            jet_1.deltaPhi(jet_2),
            jet_1.deltaR(jet_2)
        ))

        label = np.column_stack(((jet_label[:,i]*1).to_numpy(), (jet_label[:,j]*1).to_numpy()))
        output = model.predict(jet_features)

        test_labels.append(label[:,0])
        test_outputs.append(output[:,0])

test_labels, test_outputs

1749/1749 [==============================] - 3s 1ms/step


([array([0, 1, 0, ..., 0, 1, 1]),
  array([0, 1, 0, ..., 0, 1, 1]),
  array([0, 1, 0, ..., 0, 1, 1]),
  array([0, 1, 0, ..., 0, 1, 1]),
  array([0, 1, 0, ..., 0, 1, 1]),
  array([1, 0, 0, ..., 1, 0, 1]),
  array([1, 0, 0, ..., 1, 0, 1]),
  array([1, 0, 0, ..., 1, 0, 1]),
  array([1, 0, 0, ..., 1, 0, 1]),
  array([0, 1, 1, ..., 1, 0, 0]),
  array([0, 1, 1, ..., 1, 0, 0]),
  array([0, 1, 1, ..., 1, 0, 0]),
  array([0, 0, 0, ..., 0, 0, 0]),
  array([0, 0, 0, ..., 0, 0, 0]),
  array([0, 0, 1, ..., 0, 1, 0])],
 [array([1., 1., 1., ..., 1., 1., 1.], dtype=float32),
  array([1., 1., 1., ..., 1., 1., 1.], dtype=float32),
  array([1., 1., 1., ..., 1., 1., 1.], dtype=float32),
  array([1., 1., 1., ..., 1., 1., 1.], dtype=float32),
  array([1., 1., 1., ..., 1., 1., 1.], dtype=float32),
  array([1., 1., 1., ..., 1., 1., 1.], dtype=float32),
  array([1., 1., 1., ..., 1., 1., 1.], dtype=float32),
  array([1., 1., 1., ..., 1., 1., 1.], dtype=float32),
  array([1., 1., 1., ..., 1., 1., 1.], dtype=floa

In [155]:
labels = np.column_stack((labels))
outputs = np.column_stack((outputs))